In [1]:
from matplotlib.gridspec import GridSpec
from netCDF4 import Dataset
import matplotlib
import matplotlib.cm as cm 
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import matplotlib.colors
import matplotlib.colors as colors
from matplotlib.colors import LogNorm
import numpy as np
from datetime import datetime, timedelta
import datetime as dt
import xarray as xr
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
import matplotlib.colors as mcols
import glob 
import colorcet as cc
import netCDF4
import cmaps
from scipy.interpolate import interp2d
import cartopy
import cartopy.crs as ccrs
from cartopy.feature import NaturalEarthFeature
import matplotlib.gridspec as gridspec
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from pyproj import Proj
# from wrf import getvar, interplevel, to_np, latlon_coords, get_cartopy, cartopy_xlim, cartopy_ylim
from colorspacious import cspace_converter
import pathlib
from pathlib import Path
import numpy.ma as ma
from numpy import genfromtxt
import pandas as pd
import calendar
from IPython.core.pylabtools import figsize
from scipy import stats
import sys
import os

# Objective
This script just compiles all of the split/merge ARs with IVT data into one csv

In [2]:
# Update the path and pattern to match your files
# file_pattern = '/home/disk/orca/csmall3/AR_testing_research/Final_AR_Results/text_files/Total_Global_Splt_MRG_ARs_*.csv'
file_pattern = '/home/disk/orca/csmall3/AR_testing_research/Final_AR_Results/archived_text_files/Total_Global_Splt_MRG_ARs_*.csv'
file_list = glob.glob(file_pattern)

# Read and concatenate all files
combined_ivt_df = pd.concat((pd.read_csv(file) for file in file_list), ignore_index=True)

In [3]:
combined_ivt_df
#34,636 ARs for this period June 2000 to June 2024

,AR ID (Number),End Season Year,Max IVT,AR ID (string)
0,0.1,2000,619.63140,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...
1,1.1,2000,981.60516,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...
2,1.2,2000,247.71550,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...
3,2.1,2000,1043.99040,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...
4,2.2,2000,1228.74560,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...
...,...,...,...,...
34631,765.2,2004,945.25977,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...
34632,766.1,2004,407.89670,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...
34633,767.1,2004,1665.14220,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...
34634,768.1,2004,878.59460,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...


In [4]:
#save the outputs
combined_ivt_df.to_csv('/home/disk/orca/csmall3/AR_testing_research/Final_AR_Results/text_files/Total_Global_Splt_MRG_ARs.csv')

## Cleaning the MJO-Connected ARs in Pac Basin
Basically I'm going to filter through my family information now clean up the MJO Connected CSV

In [5]:
mjo_conn_ovr_df = pd.read_csv('/home/disk/orca/csmall3/AR_testing_research/Final_AR_Results/text_files/Pac_Basin_MJO_Connected_AR_Fams.csv')
mjo_conn_ovr_df = mjo_conn_ovr_df.drop(columns=['Unnamed: 0'])

In [6]:
mjo_conn_ovr_df

,AR ID,Overlap Datetimes,AR-MJO Overlap?
0,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2000-06-01T00:00:00,False
1,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2000-06-01T01:00:00,False
2,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2000-06-01T02:00:00,False
3,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2000-06-01T03:00:00,False
4,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2000-06-01T04:00:00,False
...,...,...,...
374252,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2024-06-30T19:00:00,False
374253,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2024-06-30T20:00:00,False
374254,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2024-06-30T21:00:00,False
374255,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2024-06-30T22:00:00,False


In [7]:
#filter to just get it for the families that do overlap
mjo_clean_df=mjo_conn_ovr_df.loc[mjo_conn_ovr_df['AR-MJO Overlap?'] == True].reset_index(drop=True)

In [8]:
mjo_clean_df #260 families have AR-MJO connection somewhere

,AR ID,Overlap Datetimes,AR-MJO Overlap?
0,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2000-06-24T03:00:00,True
1,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2000-07-07T05:00:00,True
2,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2000-10-10T19:00:00,True
3,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2000-11-24T12:00:00,True
4,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2001-01-28T07:00:00,True
...,...,...,...
268,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2024-01-16T12:00:00,True
269,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2024-02-08T02:00:00,True
270,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2024-03-11T15:00:00,True
271,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2024-05-19T21:00:00,True


In [9]:
#save outputs
mjo_clean_df.to_csv('/home/disk/orca/csmall3/AR_testing_research/Final_AR_Results/text_files/Pac_Basin_MJO_Connected_AR_Fams.csv')

## Clean the second level of MJO-Connected ARs
The ones that didn't meet the time criteria, but otherwise still overlap

In [2]:
mjo_def_2_df = pd.read_csv('/home/disk/orca/csmall3/AR_testing_research/Final_AR_Results/text_files/Pac_Basin_MJO_Connected_AR_Split_Merge_DEF_pt2.csv')
mjo_def_2_df = mjo_def_2_df.drop(columns=['Unnamed: 0'])

In [3]:
mjo_def_2_df

,AR ID (string),Overlap Datetimes,AR-MJO Overlap?,Max IVT,AR ID (Number),End Season Year
0,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2000-06-22T12:00:00,False,675.66003,49.10,2000
1,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2000-06-22T13:00:00,False,675.66003,49.10,2000
2,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2000-06-22T14:00:00,False,675.66003,49.10,2000
3,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2000-06-22T15:00:00,False,675.66003,49.10,2000
4,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2000-06-22T16:00:00,False,675.66003,49.10,2000
...,...,...,...,...,...,...
68004,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2024-05-24T19:00:00,False,1108.82530,623.05,2023
68005,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2024-05-24T20:00:00,False,1108.82530,623.05,2023
68006,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2024-05-24T21:00:00,False,1108.82530,623.05,2023
68007,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2024-05-24T22:00:00,False,1108.82530,623.05,2023


In [4]:
#filter to just get it for the families that do overlap
mjo_def_2_clean_df=mjo_def_2_df.loc[mjo_def_2_df['AR-MJO Overlap?'] == True].reset_index(drop=True)

In [5]:
mjo_def_2_clean_df

,AR ID (string),Overlap Datetimes,AR-MJO Overlap?,Max IVT,AR ID (Number),End Season Year
0,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2000-06-24T03:00:00,True,675.66003,49.10,2000
1,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2000-10-10T19:00:00,True,1686.10190,297.10,2000
2,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2000-11-24T12:00:00,True,1699.66520,410.10,2000
3,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2001-02-02T15:00:00,True,1863.88830,539.10,2000
4,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2001-02-06T03:00:00,True,725.14246,545.10,2000
...,...,...,...,...,...,...
205,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2024-02-08T02:00:00,True,304.85810,429.10,2023
206,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2024-03-11T23:00:00,True,1121.43850,469.20,2023
207,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2024-05-19T22:00:00,True,416.67220,607.10,2023
208,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2024-05-24T23:00:00,True,1449.56090,623.04,2023


In [7]:
#save outputs
mjo_def_2_clean_df.to_csv('/home/disk/orca/csmall3/AR_testing_research/Final_AR_Results/text_files/Pac_Basin_MJO_Connected_AR_Split_Merge_DEF_pt2.csv')

## Consolidate the MJO-Connected ARs

In [2]:
mjo_conn_1 = pd.read_csv('/home/disk/orca/csmall3/AR_testing_research/Final_AR_Results/text_files/Pac_Basin_MJO_Connected_AR_Split_Merge_DEF.csv')
mjo_conn_1 = mjo_conn_1.drop(columns=['Unnamed: 0'])

mjo_conn_2 = pd.read_csv('/home/disk/orca/csmall3/AR_testing_research/Final_AR_Results/text_files/Pac_Basin_MJO_Connected_AR_Split_Merge_DEF_pt2.csv')
mjo_conn_2 = mjo_conn_2.drop(columns=['Unnamed: 0'])

In [3]:
mjo_conn_1

,AR ID (Number),End Season Year,Max IVT,AR ID (string)
0,49.20,2000,793.52400,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...
1,77.10,2000,2551.81400,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...
2,77.20,2000,1595.32150,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...
3,77.30,2000,1422.12010,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...
4,77.40,2000,805.00740,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...
...,...,...,...,...
691,623.12,2023,2201.92820,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...
692,623.13,2023,2137.13720,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...
693,623.14,2023,1575.45810,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...
694,623.15,2023,1276.64690,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...


In [4]:
mjo_conn_2

,AR ID (string),Overlap Datetimes,AR-MJO Overlap?,Max IVT,AR ID (Number),End Season Year
0,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2000-06-22T12:00:00,False,675.66003,49.10,2000
1,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2000-06-22T13:00:00,False,675.66003,49.10,2000
2,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2000-06-22T14:00:00,False,675.66003,49.10,2000
3,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2000-06-22T15:00:00,False,675.66003,49.10,2000
4,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2000-06-22T16:00:00,False,675.66003,49.10,2000
...,...,...,...,...,...,...
72471,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2024-05-24T19:00:00,False,1108.82530,623.05,2023
72472,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2024-05-24T20:00:00,False,1108.82530,623.05,2023
72473,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2024-05-24T21:00:00,False,1108.82530,623.05,2023
72474,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2024-05-24T22:00:00,False,1108.82530,623.05,2023


In [5]:
mjo_conn_2=mjo_conn_2.loc[mjo_conn_2['AR-MJO Overlap?'] == True].reset_index(drop=True)

In [6]:
mjo_conn_2

,AR ID (string),Overlap Datetimes,AR-MJO Overlap?,Max IVT,AR ID (Number),End Season Year
0,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2000-06-24T03:00:00,True,675.66003,49.10,2000
1,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2000-10-10T19:00:00,True,1686.10190,297.10,2000
2,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2000-11-24T12:00:00,True,1699.66520,410.10,2000
3,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2001-02-02T15:00:00,True,1863.88830,539.10,2000
4,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2001-02-06T03:00:00,True,725.14246,545.10,2000
...,...,...,...,...,...,...
214,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2024-02-08T02:00:00,True,304.85810,429.10,2023
215,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2024-03-11T23:00:00,True,1121.43850,469.20,2023
216,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2024-05-19T22:00:00,True,416.67220,607.10,2023
217,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2024-05-24T23:00:00,True,1449.56090,623.04,2023


In [7]:
mjo_conn_2_clean = pd.DataFrame({'AR ID (Number)':mjo_conn_2['AR ID (Number)'],
                                'End Season Year':mjo_conn_2['End Season Year'],
                                'Max IVT':mjo_conn_2['Max IVT'],
                                'AR ID (string)':mjo_conn_2['AR ID (string)']})

In [8]:
mjo_conn_2_clean

,AR ID (Number),End Season Year,Max IVT,AR ID (string)
0,49.10,2000,675.66003,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...
1,297.10,2000,1686.10190,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...
2,410.10,2000,1699.66520,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...
3,539.10,2000,1863.88830,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...
4,545.10,2000,725.14246,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...
...,...,...,...,...
214,429.10,2023,304.85810,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...
215,469.20,2023,1121.43850,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...
216,607.10,2023,416.67220,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...
217,623.04,2023,1449.56090,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...


In [9]:
full_mjo_comb = pd.concat([mjo_conn_1, mjo_conn_2_clean]).reset_index(drop=True)

In [10]:
full_mjo_comb

,AR ID (Number),End Season Year,Max IVT,AR ID (string)
0,49.20,2000,793.5240,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...
1,77.10,2000,2551.8140,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...
2,77.20,2000,1595.3215,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...
3,77.30,2000,1422.1201,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...
4,77.40,2000,805.0074,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...
...,...,...,...,...
910,429.10,2023,304.8581,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...
911,469.20,2023,1121.4385,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...
912,607.10,2023,416.6722,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...
913,623.04,2023,1449.5609,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...


In [11]:
#save outputs
full_mjo_comb.to_csv('/home/disk/orca/csmall3/AR_testing_research/Final_AR_Results/text_files/Pac_Basin_MJO_Connected_ARs.csv')

## Consolidate Non-MJO ARs

In [14]:
file_pattern = '/home/disk/orca/csmall3/AR_testing_research/Final_AR_Results/text_files/Non_Active_MJO_Pac_Basin_ARs_*.csv'
file_list = glob.glob(file_pattern)

# Read and concatenate all files
combined_ivt_df = pd.concat((pd.read_csv(file) for file in file_list), ignore_index=True)
combined_ivt_df = combined_ivt_df.drop(columns=['Unnamed: 0'])

In [15]:
combined_ivt_df

,AR ID (string),MJO Flag,End Season Year,Max IVT,AR ID (Number)
0,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,0,2022,1705.13460,1.1
1,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,0,2022,1687.13550,1.2
2,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,0,2022,574.41254,1.3
3,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,0,2022,697.34140,1.4
4,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,0,2022,499.24140,1.5
...,...,...,...,...,...
1837,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,0,2019,334.84332,699.1
1838,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,0,2019,735.81650,702.1
1839,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,0,2019,484.52844,704.1
1840,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,0,2019,2432.54250,709.1


In [16]:
#let's reorganize so this is set up the same way
non_mjo_clean = pd.DataFrame({'AR ID (Number)':combined_ivt_df['AR ID (Number)'],
                                'End Season Year':combined_ivt_df['End Season Year'],
                                'Max IVT':combined_ivt_df['Max IVT'],
                                'AR ID (string)':combined_ivt_df['AR ID (string)']})

In [17]:
non_mjo_clean

,AR ID (Number),End Season Year,Max IVT,AR ID (string)
0,1.1,2022,1705.13460,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...
1,1.2,2022,1687.13550,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...
2,1.3,2022,574.41254,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...
3,1.4,2022,697.34140,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...
4,1.5,2022,499.24140,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...
...,...,...,...,...
1837,699.1,2019,334.84332,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...
1838,702.1,2019,735.81650,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...
1839,704.1,2019,484.52844,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...
1840,709.1,2019,2432.54250,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...


In [18]:
#save outputs
non_mjo_clean.to_csv('/home/disk/orca/csmall3/AR_testing_research/Final_AR_Results/text_files/Pac_Basin_Non_MJO_ARs.csv')

## Consolidate Landfalling MJO-Connected ARs

In [2]:
mjo_conn = pd.read_csv('/home/disk/orca/csmall3/AR_testing_research/Final_AR_Results/text_files/LF_MJO_Connected_ARs.csv')
mjo_conn = mjo_conn.drop(columns=['Unnamed: 0'])

In [3]:
mjo_conn

,AR ID (string),Overlap Datetimes,Landfall?,Max IVT,End Season Year,AR ID (Number)
0,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2000-06-24T07:00:00,False,793.5240,2000,49.20
1,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2000-06-24T08:00:00,False,793.5240,2000,49.20
2,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2000-06-24T09:00:00,False,793.5240,2000,49.20
3,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2000-06-24T10:00:00,False,793.5240,2000,49.20
4,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2000-06-24T11:00:00,False,793.5240,2000,49.20
...,...,...,...,...,...,...
99337,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2024-05-27T15:00:00,False,1108.8253,2023,623.05
99338,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2024-05-27T16:00:00,False,1108.8253,2023,623.05
99339,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2024-05-27T17:00:00,False,1108.8253,2023,623.05
99340,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2024-05-27T18:00:00,False,1108.8253,2023,623.05


In [4]:
mjo_conn=mjo_conn.loc[mjo_conn['Landfall?'] == True].reset_index(drop=True)

In [5]:
mjo_conn

,AR ID (string),Overlap Datetimes,Landfall?,Max IVT,End Season Year,AR ID (Number)
0,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2000-07-27T01:00:00,True,1595.32150,2000,77.2
1,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2000-07-28T00:00:00,True,1691.76640,2000,77.7
2,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2000-10-16T19:00:00,True,1037.16550,2000,297.3
3,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2000-10-19T19:00:00,True,1423.64220,2000,297.4
4,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2000-11-27T00:00:00,True,916.03796,2000,410.3
...,...,...,...,...,...,...
252,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2022-01-20T08:00:00,True,2158.68680,2021,373.3
253,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2022-01-24T10:00:00,True,1447.13000,2021,393.3
254,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2022-10-30T06:00:00,True,2192.27540,2022,281.1
255,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2023-02-10T01:00:00,True,1833.31200,2022,465.1


In [6]:
#save outputs
mjo_conn.to_csv('/home/disk/orca/csmall3/AR_testing_research/Final_AR_Results/text_files/LF_MJO_Connected_ARs.csv')

## Consolidate Landfalling MJO-Active ARs

In [7]:
mjo_conn = pd.read_csv('/home/disk/orca/csmall3/AR_testing_research/Final_AR_Results/text_files/LF_MJO_Active_ARs.csv')
mjo_conn = mjo_conn.drop(columns=['Unnamed: 0'])

In [8]:
mjo_conn

,AR ID (string),Overlap Datetimes,Landfall?,Max IVT,End Season Year,AR ID (Number)
0,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2000-06-14T13:00:00,False,1905.2291,2000,37.1
1,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2000-06-14T14:00:00,False,1905.2291,2000,37.1
2,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2000-06-14T15:00:00,False,1905.2291,2000,37.1
3,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2000-06-14T16:00:00,False,1905.2291,2000,37.1
4,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2000-06-14T17:00:00,False,1905.2291,2000,37.1
...,...,...,...,...,...,...
336277,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2024-05-31T02:00:00,False,582.5234,2023,633.1
336278,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2024-05-31T03:00:00,False,582.5234,2023,633.1
336279,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2024-05-31T04:00:00,False,582.5234,2023,633.1
336280,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2024-05-31T05:00:00,False,582.5234,2023,633.1


In [9]:
mjo_conn=mjo_conn.loc[mjo_conn['Landfall?'] == True].reset_index(drop=True)

In [10]:
mjo_conn

,AR ID (string),Overlap Datetimes,Landfall?,Max IVT,End Season Year,AR ID (Number)
0,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2000-06-24T06:00:00,True,1905.22910,2000,37.10
1,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2000-06-30T09:00:00,True,1755.00820,2000,37.20
2,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2000-09-09T22:00:00,True,1498.86670,2000,191.10
3,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2000-09-17T17:00:00,True,2002.69420,2000,191.12
4,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2000-09-13T01:00:00,True,1981.54740,2000,237.10
...,...,...,...,...,...,...
575,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2024-05-02T02:00:00,True,573.95807,2023,584.20
576,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2024-05-04T06:00:00,True,405.47827,2023,587.30
577,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2024-05-16T14:00:00,True,1225.15840,2023,597.30
578,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2024-05-22T03:00:00,True,610.96204,2023,597.40


In [11]:
#save outputs
mjo_conn.to_csv('/home/disk/orca/csmall3/AR_testing_research/Final_AR_Results/text_files/LF_MJO_Active_ARs.csv')

## Consolidate Landfalling Non-MJO ARs

In [12]:
mjo_conn = pd.read_csv('/home/disk/orca/csmall3/AR_testing_research/Final_AR_Results/text_files/LF_Non_MJO_ARs.csv')
mjo_conn = mjo_conn.drop(columns=['Unnamed: 0'])

In [13]:
mjo_conn

,AR ID (string),Overlap Datetimes,Landfall?,Max IVT,End Season Year,AR ID (Number)
0,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2022-06-01T00:00:00,False,1705.1346,2022,1.1
1,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2022-06-01T01:00:00,False,1705.1346,2022,1.1
2,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2022-06-01T02:00:00,False,1705.1346,2022,1.1
3,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2022-06-01T03:00:00,False,1705.1346,2022,1.1
4,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2022-06-01T04:00:00,False,1705.1346,2022,1.1
...,...,...,...,...,...,...
171695,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2020-06-30T19:00:00,False,1687.1965,2019,711.1
171696,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2020-06-30T20:00:00,False,1687.1965,2019,711.1
171697,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2020-06-30T21:00:00,False,1687.1965,2019,711.1
171698,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2020-06-30T22:00:00,False,1687.1965,2019,711.1


In [14]:
mjo_conn=mjo_conn.loc[mjo_conn['Landfall?'] == True].reset_index(drop=True)

In [15]:
mjo_conn

,AR ID (string),Overlap Datetimes,Landfall?,Max IVT,End Season Year,AR ID (Number)
0,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2022-06-04T08:00:00,True,1705.13460,2022,1.10
1,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2022-06-08T03:00:00,True,1687.13550,2022,1.20
2,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2022-08-27T14:00:00,True,2099.82200,2022,122.09
3,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2022-09-04T06:00:00,True,1319.06960,2022,122.12
4,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2022-09-04T08:00:00,True,1401.68420,2022,196.10
...,...,...,...,...,...,...
307,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2019-11-16T19:00:00,True,1705.68650,2019,233.18
308,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2019-12-10T22:00:00,True,1953.45040,2019,312.70
309,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2019-12-12T16:00:00,True,641.44366,2019,312.80
310,/home/orca/bkerns/projects/mjo_lpt_and_ar/trac...,2020-06-15T00:00:00,True,1821.59230,2019,659.20


In [16]:
#save outputs
mjo_conn.to_csv('/home/disk/orca/csmall3/AR_testing_research/Final_AR_Results/text_files/LF_Non_MJO_ARs.csv')